In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

### Table of Contents

* [modify survey data](#modify_survey_data)
    * [read raw data](#read_raw_data)
    * [build before_transfers and 'access_mode' from raw access mode fields](#build_access_mode)
    * [build after_transfers and 'egress_mode' from raw access mode fields](#build_egress_mode)
    * [deal with 'hispanic' and 'ethnicity/race'](#race)
    * [deal with trip purpose](#trip_purp)
    * [code home lat/lon based on zipcode](#home_lat_lon)
    * [code board/alight station lat/lon](#station_lat_lon)
    * [update 'weight'](#weight)
    * [impute year_born from 'age group'](#age)
    * [export survey data](#survey_export)

* [build standard dictionary](#standard_dict)
    * [read raw variable dictionary 'Field Guide'](#raw_dict)
    * [add rows to the dictionary for the new fields added to the survey data](#add_row)
    * [add default fields in the standard dictionary](#add_fields)
    * [check consistency between values in survey data and in the dictionary](#check)
    * [export raw standard dictionary](#export_dict)
    
* [build canonical route crosswalk](#canonical_route)

## modify survey data <a class="anchor" id="modify_survey_data"></a>

### read raw data  <a class="anchor" id="read_raw_data"></a>

In [2]:
df_raw = pd.read_excel(r'M:\Data\OnBoard\Data and Reports\Capitol Corridor\OD Survey 2019\CAPCO19 Data-For MTC.xlsx',
                       sheet_name='Data')
print('read {} records, with {} unique CCGID'.format(df_raw.shape[0], len(df_raw.CCGID.unique())))

df = df_raw.copy()
print(list(df))

read 2406 records, with 2406 unique CCGID
['RESPNUM', 'CCGID', 'TRAIN', 'INTDAY', 'INTDATE', 'PERIOD', 'LANGUAGE', 'Q1A', 'BOARD', 'Q1B', 'ALIGHT', 'Q2A_1', 'Q2A_2', 'Q2A_3', 'Q2A_4', 'Q2A_5', 'Q2A_6', 'Q2B_1', 'Q2B_2', 'Q2B_3', 'Q2B_4', 'Q2B_5', 'Q2B_6', 'Q3', 'Q4', 'Q5_1', 'Q5_2', 'Q5_3', 'Q5_4', 'Q6_1', 'Q6_2', 'Q6_3', 'Q6_4', 'Q7', 'Q8_1', 'Q8_2', 'Q8_3', 'Q8_4', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13_1', 'Q13_2', 'Q13_3', 'Q13_4', 'Q15_1', 'Q15_2', 'Q15_3', 'Q15_4', 'Q16_1', 'Q16_2', 'Q16_3', 'Q16_4', 'Q17', 'Q18', 'Q19', 'Unnamed: 57', 'Q20_1', 'Q20_2', 'Q20_3', 'Q20_4', 'Q21', 'Q22', 'CITY', 'CITY_CODE', 'COUNTY', 'COUNTY_CODE ', 'STATE', 'STATE_CODE ', 'COUNTRY', 'WEIGHT']


### build before_transfers and 'access_mode' from raw access mode fields Q2A_1-Q2A_6 <a class="anchor" id="build_access_mode"></a>

In [3]:
"""
row dictionary of Q2A_1-Q2A_6 / Q2B_1-Q2B_6:

   {1: 'Dropped off/Picked up', 
    2: 'Drove alone', 
    3: 'Carpool', 
    4: 'Taxi/Uber/Lyft', 
    5: 'BART', 
    6: 'Caltrain', 
    7: 'Light rail (VTA, Sacramento RT)', 
    8: 'Amtrak thruway bus', 
    9: 'Amtrak long distance train', 
    10: 'Bus transit', 
    11: 'Walked', 
    12: 'Bike', 
    13: 'Electric Scooter/Scooter', 
    14: 'Other (Specify)' 
    }
"""
## create fields

for colname in ['access_mode', 'egress_mode',
                'first_route_before_survey_board', 'second_route_before_survey_board', 'third_route_before_survey_board',
                'first_route_after_survey_alight', 'second_route_after_survey_alight', 'third_route_after_survey_alight']:
    df[colname] = np.nan
    
# first, error in CCGID 63 Q2A_3, update to nan
display(df.loc[df.CCGID == 63][['CCGID', 'Q2A_1', 'Q2A_2', 'Q2A_3', 'Q2A_4', 'Q2A_5', 'Q2A_6']])
df.loc[df.CCGID == 63, 'Q2A_3'] = np.nan

,CCGID,Q2A_1,Q2A_2,Q2A_3,Q2A_4,Q2A_5,Q2A_6
62,63,4,NaN,11+H13:H32,NaN,NaN,NaN


In [4]:
# 1. for responses with Q2A_1 only and Q2A_1 is not public transit, set Q2A_1 as access mode, and no before-transfer
idx_1 = ((df['Q2A_1'] < 5) | (df['Q2A_1'] > 10)) & (     # Q2A_1 not public transit
    df['Q2A_1'].notnull() &
    df['Q2A_2'].isnull() & 
    df['Q2A_3'].isnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_1 find {} rows, {:.1%} of total'.format(idx_1.sum(), idx_1.sum()/df.shape[0]))
df.loc[idx_1, 'access_mode'] = df['Q2A_1']

# 2. for responses with Q2A_1 only and Q2A_1 is public transit, set access mode as Missing, and Q2A_1 as before-transfer
idx_2 = ((df['Q2A_1'] > 4) & (df['Q2A_1'] < 11)) & (    # Q2A_1 public transit
    df['Q2A_1'].notnull() &
    df['Q2A_2'].isnull() & 
    df['Q2A_3'].isnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_2 find {} rows, {:.1%} of total'.format(idx_2.sum(), idx_2.sum()/df.shape[0]))
df.loc[idx_2, 'access_mode'] = 'Missing'
df.loc[idx_2, 'first_route_before_survey_board'] = df['Q2A_1']

# 3. for responses with Q2A_1 and Q2A_2 only, and Q2A_1 is public transit, Q2A_2 is not public transit, 
# set access mode as Missing, and Q2A_1 as before-transfer
idx_3 = ((df['Q2A_1'] > 4) & (df['Q2A_1'] < 11)) & (    # Q2A_1 public transit
         (df['Q2A_2'] < 5) | (df['Q2A_2'] > 10)) & (    # Q2A_2 not public transit
    df['Q2A_1'].notnull() & 
    df['Q2A_2'].notnull() & 
    df['Q2A_3'].isnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_3 find {} rows, {:.1%} of total'.format(idx_3.sum(), idx_3.sum()/df.shape[0]))
df.loc[idx_3, 'access_mode'] = 'Missing'
df.loc[idx_3, 'first_route_before_survey_board'] = df['Q2A_1']

# 4. for responses with Q2A_1 and Q2A_2 only, and both are public transit 
# set access mode as Missing, and Q2A_1, Q2A_2 as before-transfers
idx_4 = ((df['Q2A_1'] > 4) & (df['Q2A_1'] < 11)) & (    # Q2A_1 public transit
         (df['Q2A_2'] > 4) & (df['Q2A_2'] < 11)) & (    # Q2A_2 public transit
    df['Q2A_1'].notnull() & 
    df['Q2A_2'].notnull() & 
    df['Q2A_3'].isnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_4 find {} rows, {:.1%} of total'.format(idx_4.sum(), idx_4.sum()/df.shape[0]))
df.loc[idx_4, 'access_mode'] = 'Missing'
df.loc[idx_4, 'first_route_before_survey_board'] = df['Q2A_1']
df.loc[idx_4, 'second_route_before_survey_board'] = df['Q2A_2']

# 5. for responses with Q2A_1 and Q2A_2 only, and both are not public transit
# set access mode as Q2A_1, and no before-transfer
idx_5 = ((df['Q2A_1'] < 5) | (df['Q2A_1'] > 10)) & (    # Q2A_1 not public transit
         (df['Q2A_2'] < 5) | (df['Q2A_2'] > 10)) & (    # Q2A_2 not public transit
    df['Q2A_1'].notnull() & 
    df['Q2A_2'].notnull() & 
    df['Q2A_3'].isnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_5 find {} rows, {:.1%} of total'.format(idx_5.sum(), idx_5.sum()/df.shape[0]))
df.loc[idx_5, 'access_mode'] =  df['Q2A_1']

# 6. for responses with Q2A_1 and Q2A_2 only, and Q2A_1 is not public transit, Q2A_2 is public transit,
# set access mode as Q2A_1, Q2A_2 is before-transfer
idx_6 = ((df['Q2A_1'] < 5) | (df['Q2A_1'] > 10)) & (    # Q2A_1 not public transit
         (df['Q2A_2'] > 4) & (df['Q2A_2'] < 11)) & (    # Q2A_2 public transit
    df['Q2A_1'].notnull() & 
    df['Q2A_2'].notnull() & 
    df['Q2A_3'].isnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_6 find {} rows, {:.1%} of total'.format(idx_6.sum(), idx_6.sum()/df.shape[0]))
df.loc[idx_6, 'access_mode'] =  df['Q2A_1']
df.loc[idx_6, 'first_route_before_survey_board'] = df['Q2A_2']

# 7. for responses with Q2A_1, Q2A_2, and Q2A_3 only, and all three are public transit,
# set access mode as Missing, and Q2A_1, Q2A_2, Q2A_3 as before-transfers
idx_7 = ((df['Q2A_1'] > 4) & (df['Q2A_1'] < 11)) & (    # Q2A_1 public transit
         (df['Q2A_2'] > 4) & (df['Q2A_2'] < 11)) & (    # Q2A_2 public transit
         (df['Q2A_3'] > 4) & (df['Q2A_3'] < 11)) & (    # Q2A_3 public transit
    df['Q2A_1'].notnull() & 
    df['Q2A_2'].notnull() & 
    df['Q2A_3'].notnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_7 find {} rows, {:.1%} of total'.format(idx_7.sum(), idx_7.sum()/df.shape[0]))
df.loc[idx_7, 'access_mode'] =  'Missing'
df.loc[idx_7, 'first_route_before_survey_board'] = df['Q2A_1']
df.loc[idx_7, 'second_route_before_survey_board'] = df['Q2A_2']
df.loc[idx_7, 'third_route_before_survey_board'] = df['Q2A_3']

# 8. for responses with Q2A_1, Q2A_2, and Q2A_3 only, and all three are not public transit,
# set access mode as Q2A_1, and no before-transfer
idx_8 = ((df['Q2A_1'] < 5) | (df['Q2A_1'] > 10)) & (    # Q2A_1 not public transit
         (df['Q2A_2'] < 5) | (df['Q2A_2'] > 10)) & (    # Q2A_2 not public transit
         (df['Q2A_3'] < 5) | (df['Q2A_3'] > 10)) & (    # Q2A_3 not public transit
    df['Q2A_1'].notnull() & 
    df['Q2A_2'].notnull() & 
    df['Q2A_3'].notnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_8 find {} rows, {:.1%} of total'.format(idx_8.sum(), idx_8.sum()/df.shape[0]))
df.loc[idx_8, 'access_mode'] =  df['Q2A_1']

# 9. for responses with Q2A_1, Q2A_2, and Q2A_3 only, and Q2A_1 is public transit, Q2A_2 and Q2A_3 not public transit,
# set access mode as Missing, and Q2A_1 as before-transfer
idx_9 = ((df['Q2A_1'] > 4) & (df['Q2A_1'] < 11)) & (    # Q2A_1 public transit
         (df['Q2A_2'] < 5) | (df['Q2A_2'] > 10)) & (    # Q2A_2 not public transit
         (df['Q2A_3'] < 5) | (df['Q2A_3'] > 10)) & (    # Q2A_3 not public transit
    df['Q2A_1'].notnull() & 
    df['Q2A_2'].notnull() & 
    df['Q2A_3'].notnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_9 find {} rows, {:.1%} of total'.format(idx_9.sum(), idx_9.sum()/df.shape[0]))
df.loc[idx_9, 'access_mode'] = 'Missing'
df.loc[idx_9, 'first_route_before_survey_board'] = df['Q2A_1']

# 10. for responses with Q2A_1, Q2A_2, and Q2A_3 only, and Q2A_1, Q2A_2 are public transit, Q2A_3 not public transit,
# set access mode as Missing, and Q2A_1, Q2A_2 as before-transfers
idx_10 = ((df['Q2A_1'] > 4) & (df['Q2A_1'] < 11)) & (    # Q2A_1 public transit
          (df['Q2A_2'] > 4) & (df['Q2A_2'] < 11)) & (    # Q2A_2 public transit
          (df['Q2A_3'] < 5) | (df['Q2A_3'] > 10)) & (    # Q2A_3 not public transit
    df['Q2A_1'].notnull() & 
    df['Q2A_2'].notnull() & 
    df['Q2A_3'].notnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_10 find {} rows, {:.1%} of total'.format(idx_10.sum(), idx_10.sum()/df.shape[0]))
df.loc[idx_10, 'access_mode'] =  'Missing'
df.loc[idx_10, 'first_route_before_survey_board'] = df['Q2A_1']
df.loc[idx_10, 'second_route_before_survey_board'] = df['Q2A_2']

# 11. for responses with Q2A_1, Q2A_2, and Q2A_3 only, and Q2A_1 is not public transit, Q2A_2/Q2A_3 are public transit,
# set access mode as Q2A_1, and Q2A_2, Q2A_3 as before-transfers
idx_11 = ((df['Q2A_1'] < 5) | (df['Q2A_1'] > 10)) & (    # Q2A_1 not public transit
          (df['Q2A_2'] > 4) & (df['Q2A_2'] < 11)) & (    # Q2A_2 public transit
          (df['Q2A_3'] > 4) & (df['Q2A_3'] < 11)) & (    # Q2A_3 public transit
    df['Q2A_1'].notnull() & 
    df['Q2A_2'].notnull() & 
    df['Q2A_3'].notnull() & 
    df['Q2A_4'].isnull() & 
    df['Q2A_5'].isnull() & 
    df['Q2A_6'].isnull())

print('idx_11 find {} rows, {:.1%} of total'.format(idx_11.sum(), idx_11.sum()/df.shape[0]))
df.loc[idx_11, 'access_mode'] = df['Q2A_1']
df.loc[idx_11, 'first_route_before_survey_board'] = df['Q2A_2']
df.loc[idx_11, 'second_route_before_survey_board'] = df['Q2A_3']

# for the rest (access_mode still is nan), if Q2A_1 is not public transit, set access mode as Q2A_1, and no before-transfer
idx_12 = ((df['Q2A_1'] < 5) | (df['Q2A_1'] > 10)) & (    # Q2A_1 not public transit
          df.access_mode.isnull())

print('idx_12 find {} rows, {:.1%} of total'.format(idx_12.sum(), idx_12.sum()/df.shape[0]))

df.loc[idx_12, 'access_mode'] = df['Q2A_1']
# df.loc[idx_12][['CCGID', 'Q2A_1', 'Q2A_2', 'Q2A_3', 'Q2A_4', 'Q2A_5', 'Q2A_6']].dropna(how='all', axis=1).drop_duplicates()

# examine the remaining access_mode.isnull()
display(df.loc[df.access_mode.isnull()][['CCGID', 'Q2A_1', 'Q2A_2', 'Q2A_3', 'Q2A_4', 'Q2A_5', 'Q2A_6']].dropna(how='all', axis=1))

idx_1 find 1923 rows, 79.9% of total
idx_2 find 303 rows, 12.6% of total
idx_3 find 18 rows, 0.7% of total
idx_4 find 4 rows, 0.2% of total
idx_5 find 87 rows, 3.6% of total
idx_6 find 27 rows, 1.1% of total
idx_7 find 1 rows, 0.0% of total
idx_8 find 11 rows, 0.5% of total
idx_9 find 3 rows, 0.1% of total
idx_10 find 0 rows, 0.0% of total
idx_11 find 1 rows, 0.0% of total
idx_12 find 25 rows, 1.0% of total


,CCGID,Q2A_1,Q2A_2,Q2A_3,Q2A_4
967,969,8,4.0,9,10.0
1120,1122,10,1.0,4,12.0
2037,2039,10,11.0,7,NaN


In [5]:
# for CCGID 969 (Q2A_1 'Amtrak long-distance bus', Q2A_2 'taxi/uber/lyft', Q2A_3 'Amtrak train', Q2A_4 'bus', 
# set Q2A_1 as before-transfer, and access_mode as Missing
df.loc[df.CCGID==969, 'access_mode'] = 'Missing'
df.loc[df.CCGID==969, 'first_route_before_survey_board'] = df['Q2A_1']

# for CCGID 1122 (Q2A_1 'bus', Q2A_2 'drop off/pick up', Q2A_3 'taxi/uber/lyft', Q2A_4 'bike', 
# set Q2A_1 as before-transfer, and access_mode as Missing
df.loc[df.CCGID==1122, 'access_mode'] = 'Missing'
df.loc[df.CCGID==1122, 'first_route_before_survey_board'] = df['Q2A_1']

# for CCGID 2039 (Q2A_1 'bus', Q2A_2 'walk', Q2A_3 'light rail' 
# set Q2A_1, Q2A_3 as before-transfers, and access_mode as Missing
df.loc[df.CCGID==2039, 'access_mode'] = 'Missing'
df.loc[df.CCGID==2039, 'first_route_before_survey_board'] = df['Q2A_1']
df.loc[df.CCGID==2039, 'second_route_before_survey_board'] = df['Q2A_3']

# check there is no row with access_mode.isnull()
print('{} rows is missing access_mode'.format(df.access_mode.isnull().sum()))

0 rows is missing access_mode


### build after_transfers and 'egress_mode' from raw access mode fields Q2B_1-Q2B_6  <a class="anchor" id="build_egress_mode"></a>

In [6]:
# 1. for responses with Q2B_1 only and Q2B_1 is not public transit, set Q2B_1 as egress mode, and no after-transfer
idx_1 = ((df['Q2B_1'] < 5) | (df['Q2B_1'] > 10)) & (     # Q2B_1 not public transit
    df['Q2B_1'].notnull() &
    df['Q2B_2'].isnull() & 
    df['Q2B_3'].isnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_1 find {} rows, {:.1%} of total'.format(idx_1.sum(), idx_1.sum()/df.shape[0]))
df.loc[idx_1, 'egress_mode'] = df['Q2B_1']

# 2. for responses with Q2B_1 only and Q2B_1 is public transit, set egress mode as Missing, and Q2B_1 as after-transfer
idx_2 = ((df['Q2B_1'] > 4) & (df['Q2B_1'] < 11)) & (    # Q2B_1 public transit
    df['Q2B_1'].notnull() &
    df['Q2B_2'].isnull() & 
    df['Q2B_3'].isnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_2 find {} rows, {:.1%} of total'.format(idx_2.sum(), idx_2.sum()/df.shape[0]))
df.loc[idx_2, 'egress_mode'] = 'Missing'
df.loc[idx_2, 'first_route_after_survey_alight'] = df['Q2B_1']

# 3. for responses with Q2B_1 and Q2B_2 only, and Q2B_1 is public transit, Q2B_2 is not public transit, 
# set egress mode as Q2B_2, and Q2B_1 as after-transfer
idx_3 = ((df['Q2B_1'] > 4) & (df['Q2B_1'] < 11)) & (    # Q2B_1 public transit
         (df['Q2B_2'] < 5) | (df['Q2B_2'] > 10)) & (    # Q2B_2 not public transit
    df['Q2B_1'].notnull() & 
    df['Q2B_2'].notnull() & 
    df['Q2B_3'].isnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_3 find {} rows, {:.1%} of total'.format(idx_3.sum(), idx_3.sum()/df.shape[0]))
df.loc[idx_3, 'egress_mode'] = df['Q2B_2']
df.loc[idx_3, 'first_route_after_survey_alight'] = df['Q2B_1']

# 4. for responses with Q2B_1 and Q2B_2 only, and both are public transit 
# set egress mode as Missing, and Q2B_1, Q2B_2 as after-transfers
idx_4 = ((df['Q2B_1'] > 4) & (df['Q2B_1'] < 11)) & (    # Q2B_1 public transit
         (df['Q2B_2'] > 4) & (df['Q2B_2'] < 11)) & (    # Q2B_2 public transit
    df['Q2B_1'].notnull() & 
    df['Q2B_2'].notnull() & 
    df['Q2B_3'].isnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_4 find {} rows, {:.1%} of total'.format(idx_4.sum(), idx_4.sum()/df.shape[0]))
df.loc[idx_4, 'egress_mode'] = 'Missing'
df.loc[idx_4, 'first_route_after_survey_alight'] = df['Q2B_1']
df.loc[idx_4, 'second_route_after_survey_alight'] = df['Q2B_2']

# 5. for responses with Q2B_1 and Q2B_2 only, and both are not public transit
# set egress mode as Q2B_1, and no after-transfer
idx_5 = ((df['Q2B_1'] < 5) | (df['Q2B_1'] > 10)) & (    # Q2B_1 not public transit
         (df['Q2B_2'] < 5) | (df['Q2B_2'] > 10)) & (    # Q2B_2 not public transit
    df['Q2B_1'].notnull() & 
    df['Q2B_2'].notnull() & 
    df['Q2B_3'].isnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_5 find {} rows, {:.1%} of total'.format(idx_5.sum(), idx_5.sum()/df.shape[0]))
df.loc[idx_5, 'egress_mode'] =  df['Q2B_1']

# 6. for responses with Q2B_1 and Q2B_2 only, and Q2B_1 is not public transit, Q2B_2 is public transit,
# set egress mode as Missing, Q2B_2 is after-transfer
idx_6 = ((df['Q2B_1'] < 5) | (df['Q2B_1'] > 10)) & (    # Q2B_1 not public transit
         (df['Q2B_2'] > 4) & (df['Q2B_2'] < 11)) & (    # Q2B_2 public transit
    df['Q2B_1'].notnull() & 
    df['Q2B_2'].notnull() & 
    df['Q2B_3'].isnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_6 find {} rows, {:.1%} of total'.format(idx_6.sum(), idx_6.sum()/df.shape[0]))
df.loc[idx_6, 'egress_mode'] =  'Missing'
df.loc[idx_6, 'first_route_after_survey_alight'] = df['Q2B_2']

# 7. for responses with Q2B_1, Q2B_2, and Q2B_3 only, and all three are public transit,
# set egress mode as Missing, and Q2B_1, Q2B_2, Q2B_3 as after-transfers
idx_7 = ((df['Q2B_1'] > 4) & (df['Q2B_1'] < 11)) & (    # Q2B_1 public transit
         (df['Q2B_2'] > 4) & (df['Q2B_2'] < 11)) & (    # Q2B_2 public transit
         (df['Q2B_3'] > 4) & (df['Q2B_3'] < 11)) & (    # Q2B_3 public transit
    df['Q2B_1'].notnull() & 
    df['Q2B_2'].notnull() & 
    df['Q2B_3'].notnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_7 find {} rows, {:.1%} of total'.format(idx_7.sum(), idx_7.sum()/df.shape[0]))
df.loc[idx_7, 'egress_mode'] =  'Missing'
df.loc[idx_7, 'first_route_after_survey_alight'] = df['Q2B_1']
df.loc[idx_7, 'second_route_after_survey_alight'] = df['Q2B_2']
df.loc[idx_7, 'third_route_after_survey_alight'] = df['Q2B_3']

# 8. for responses with Q2B_1, Q2B_2, and Q2B_3 only, and all three are not public transit,
# set egress mode as Q2B_1, and no after-transfer
idx_8 = ((df['Q2B_1'] < 5) | (df['Q2B_1'] > 10)) & (    # Q2B_1 not public transit
         (df['Q2B_2'] < 5) | (df['Q2B_2'] > 10)) & (    # Q2B_2 not public transit
         (df['Q2B_3'] < 5) | (df['Q2B_3'] > 10)) & (    # Q2B_3 not public transit
    df['Q2B_1'].notnull() & 
    df['Q2B_2'].notnull() & 
    df['Q2B_3'].notnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_8 find {} rows, {:.1%} of total'.format(idx_8.sum(), idx_8.sum()/df.shape[0]))
df.loc[idx_8, 'egress_mode'] = df['Q2B_1']

# 9. for responses with Q2B_1, Q2B_2, and Q2B_3 only, and Q2B_1 is public transit, Q2B_2 and Q2B_3 not public transit,
# set egress mode as Q2B_2, and Q2B_1 as after-transfer
idx_9 = ((df['Q2B_1'] > 4) & (df['Q2B_1'] < 11)) & (    # Q2B_1 public transit
         (df['Q2B_2'] < 5) | (df['Q2B_2'] > 10)) & (    # Q2B_2 not public transit
         (df['Q2B_3'] < 5) | (df['Q2B_3'] > 10)) & (    # Q2B_3 not public transit
    df['Q2B_1'].notnull() & 
    df['Q2B_2'].notnull() & 
    df['Q2B_3'].notnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_9 find {} rows, {:.1%} of total'.format(idx_9.sum(), idx_9.sum()/df.shape[0]))
df.loc[idx_9, 'egress_mode'] = df['Q2B_2']
df.loc[idx_9, 'first_route_after_survey_alight'] = df['Q2B_1']

# 10. for responses with Q2B_1, Q2B_2, and Q2B_3 only, and Q2B_1, Q2B_2 are public transit, Q2B_3 not public transit,
# set egress mode as Q2B_3, and Q2B_1, Q2B_2 as after-transfers
idx_10 = ((df['Q2B_1'] > 4) & (df['Q2B_1'] < 11)) & (    # Q2B_1 public transit
          (df['Q2B_2'] > 4) & (df['Q2B_2'] < 11)) & (    # Q2B_2 public transit
          (df['Q2B_3'] < 5) | (df['Q2B_3'] > 10)) & (    # Q2B_3 not public transit
    df['Q2B_1'].notnull() & 
    df['Q2B_2'].notnull() & 
    df['Q2B_3'].notnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_10 find {} rows, {:.1%} of total'.format(idx_10.sum(), idx_10.sum()/df.shape[0]))
df.loc[idx_10, 'egress_mode'] = df['Q2B_3']
df.loc[idx_10, 'first_route_after_survey_alight'] = df['Q2B_1']
df.loc[idx_10, 'second_route_after_survey_alight'] = df['Q2B_2']

# 11. for responses with Q2B_1, Q2B_2, and Q2B_3 only, and Q2B_1 is not public transit, Q2B_2/Q2B_3 are public transit,
# set egress mode as Q2B_1, and no after-transfer
idx_11 = ((df['Q2B_1'] < 5) | (df['Q2B_1'] > 10)) & (    # Q2B_1 not public transit
          (df['Q2B_2'] > 4) & (df['Q2B_2'] < 11)) & (    # Q2B_2 public transit
          (df['Q2B_3'] > 4) & (df['Q2B_3'] < 11)) & (    # Q2B_3 public transit
    df['Q2B_1'].notnull() & 
    df['Q2B_2'].notnull() & 
    df['Q2B_3'].notnull() & 
    df['Q2B_4'].isnull() & 
    df['Q2B_5'].isnull() & 
    df['Q2B_6'].isnull())

print('idx_11 find {} rows, {:.1%} of total'.format(idx_11.sum(), idx_11.sum()/df.shape[0]))
df.loc[idx_11, 'egress_mode'] = df['Q2B_1']

# for the rest (egress_mode still is nan), if Q2B_1 is not public transit, set egress mode as Q2B_1, and no after-transfer
idx_12 = ((df['Q2B_1'] < 5) | (df['Q2B_1'] > 10)) & (    # Q2B_1 not public transit
          df.egress_mode.isnull())

print('idx_12 find {} rows, {:.1%} of total'.format(idx_12.sum(), idx_12.sum()/df.shape[0]))

df.loc[idx_12, 'egress_mode'] = df['Q2B_1']
# df.loc[idx_12][['CCGID', 'Q2B_1', 'Q2B_2', 'Q2B_3', 'Q2B_4', 'Q2B_5', 'Q2B_6']].dropna(how='all', axis=1).drop_duplicates()

# examine the remaining egress_mode.isnull()
display(df.loc[df.egress_mode.isnull()][['CCGID', 'Q2B_1', 'Q2B_2', 'Q2B_3', 'Q2B_4', 'Q2B_5', 'Q2B_6']].dropna(how='all', axis=1))

idx_1 find 1886 rows, 78.4% of total
idx_2 find 329 rows, 13.7% of total
idx_3 find 48 rows, 2.0% of total
idx_4 find 6 rows, 0.2% of total
idx_5 find 64 rows, 2.7% of total
idx_6 find 28 rows, 1.2% of total
idx_7 find 1 rows, 0.0% of total
idx_8 find 12 rows, 0.5% of total
idx_9 find 5 rows, 0.2% of total
idx_10 find 3 rows, 0.1% of total
idx_11 find 0 rows, 0.0% of total
idx_12 find 20 rows, 0.8% of total


,CCGID,Q2B_1,Q2B_2,Q2B_3,Q2B_4
441,443,5,7.0,11.0,2.0
466,468,7,11.0,10.0,NaN
570,572,10,11.0,12.0,13.0
1657,1659,6,1.0,10.0,NaN


In [7]:
# for CCGID 443 (Q2B_1 'BART', Q2B_2 'Light rail', Q2B_3 'walk', Q2B_4 'drove-alone')
# set Q2B_1, Q2B_2 as after-transfer, and egress_mode as Q2B_3
df.loc[df.CCGID==443, 'egress_mode'] = df['Q2B_3']
df.loc[df.CCGID==443, 'first_route_after_survey_alight'] = df['Q2B_1']
df.loc[df.CCGID==443, 'second_route_after_survey_alight'] = df['Q2B_2']

# for CCGID 468 (Q2B_1 'light rail', Q2B_2 'walk', Q2B_3 'bus')
# set Q2B_1 and Q2B_3 as after-transfer, and egress_mode as Missing
df.loc[df.CCGID==468, 'egress_mode'] = 'Missing'
df.loc[df.CCGID==468, 'first_route_after_survey_alight'] = df['Q2B_1']
df.loc[df.CCGID==468, 'second_route_after_survey_alight'] = df['Q2B_3']

# for CCGID 572 (Q2B_1 'bus', Q2B_2 'walk', Q2B_3 'bike', Q2B_4 'scooter') 
# set Q2B_1 as after-transfer, and egress_mode as Q2B_2
df.loc[df.CCGID==572, 'egress_mode'] = df['Q2B_2']
df.loc[df.CCGID==572, 'first_route_after_survey_alight'] = df['Q2B_1']

# for CCGID 1659 (Q2B_1 'Caltrain', Q2B_2 'drop off/pick up', Q2B_3 'bus')
# set Q2B_1 as after-transfer, and egress_mode as Q2B_2
df.loc[df.CCGID==1659, 'egress_mode'] = df['Q2B_2']
df.loc[df.CCGID==1659, 'first_route_after_survey_alight'] = df['Q2B_1']

# check there is no row with egress_mode.isnull()
print('{} rows is missing egress_mode'.format(df.egress_mode.isnull().sum()))

0 rows is missing egress_mode


### deal with 'hispanic' and 'ethnicity/race'  <a class="anchor" id="race"></a>

In [8]:
race_dict = {1: 'White',
             2: 'Black/African American',
             3: 'Asian/Pacific Islander',
             4: 'Hispanic/Latino',
             5: 'American Indian/Alaskan Native',
             6: 'Other',
             7: 'Other',
             8: 'Other',
             9: 'Mixed',
             10: 'Middle Eastern',
             11: 'East Indian/Pakistani',
             0: 'NA'}

for i in ['Q20_1', 'Q20_2', 'Q20_3', 'Q20_4']:
    df[i] = df[i].fillna(10)
    df[i].replace(to_replace = ' ', value = 10, inplace=True)
    df[i] = df[i].apply(lambda x: int(x))
    df[i+'_temp'] = df[i].map(race_dict)
    
df['race_concat'] = df['Q20_1_temp'] + '_' + df['Q20_2_temp'] + '_' + df['Q20_3_temp'] + '_' + df['Q20_4_temp']
print(list(df['race_concat'].unique()))

['Hispanic/Latino_Middle Eastern_Middle Eastern_Middle Eastern', 'White_Middle Eastern_Middle Eastern_Middle Eastern', 'Asian/Pacific Islander_Middle Eastern_Middle Eastern_Middle Eastern', 'Black/African American_Middle Eastern_Middle Eastern_Middle Eastern', 'Other_Middle Eastern_Middle Eastern_Middle Eastern', 'White_Hispanic/Latino_Middle Eastern_Middle Eastern', 'Mixed_Middle Eastern_Middle Eastern_Middle Eastern', 'White_Black/African American_Middle Eastern_Middle Eastern', 'White_Hispanic/Latino_American Indian/Alaskan Native_Middle Eastern', 'NA_Middle Eastern_Middle Eastern_Middle Eastern', 'White_Black/African American_Asian/Pacific Islander_Middle Eastern', 'American Indian/Alaskan Native_Middle Eastern_Middle Eastern_Middle Eastern', 'White_Black/African American_American Indian/Alaskan Native_Middle Eastern', 'Asian/Pacific Islander_Hispanic/Latino_Middle Eastern_Middle Eastern', 'White_Black/African American_Asian/Pacific Islander_Hispanic/Latino', 'Black/African America

In [9]:
# create 'hispanic' field
df['hispanic'] = 'NO'
df.loc[df.race_concat.str.contains('Hispanic',na=False), 'hispanic'] = 'YES'
df.hispanic.value_counts()

NO     2142
YES     264
Name: hispanic, dtype: int64

In [10]:
# create race_dmy_xx

df['race_dmy_ind'] = 0
df.loc[df.race_concat.str.contains('American Indian/Alaskan Native',na=False), 'race_dmy_ind'] = 1

df['race_dmy_hwi'] = 0
df.loc[df.race_concat.str.contains('Native Hawaiian/Pacific Islander',na=False), 'race_dmy_hwi'] = 1

df['race_dmy_blk'] = 0
df.loc[df.race_concat.str.contains('Black/African American',na=False), 'race_dmy_blk'] = 1

df['race_dmy_wht'] = 0
df.loc[df.race_concat.str.contains('White',na=False), 'race_dmy_wht'] = 1

df['race_dmy_asn'] = 0
df.loc[df.race_concat.str.contains('Asian',na=False) | df.race_concat.str.contains('East Indian/Pakistani',na=False), 'race_dmy_asn'] = 1

df['race_dmy_mdl_estn'] = 0
df.loc[df.race_concat.str.contains('Middle Eastern',na=False), 'race_dmy_mdl_estn'] = 1

# drop temp fields
df.drop(columns = ['Q20_1_temp', 'Q20_2_temp', 'Q20_3_temp', 'Q20_4_temp', 'race_concat'], inplace=True)

### deal with trip purpose <a class="anchor" id="trip_purp"></a>

In [11]:
trip_purp_dict = {
    '1': 'Commute to/from work', 
    '2': 'Business travel', 
    '3': 'Travel to/from school', 
    '4': 'Leisure/Recreation', 
    '5': 'Visit family/friends', 
    '6': 'Vacation', 
    '7': 'Other', 
    '8': 'Personal / Family business', 
    '9': 'Travel to or from school', 
    '10': 'Other (specify)', 
    '11': 'School/ Group Trip', 
    '12': 'Church/volunteering/political', 
    '13': 'Just to enjoy the train/Outing to ride train', 
    '14': 'Moving/traveling between homes', 
    '15': 'Going Home', 
    '16': 'Airport trip'}

for colname in ['Q8_1','Q8_2','Q8_3','Q8_4']:
    df[colname].fillna(0, inplace=True)
    df[colname] = df[colname].apply(lambda x: str(int(x)))
    df[colname] = df[colname].map(trip_purp_dict)

# print out all possible combinations in the data
display(df[['Q8_1','Q8_2','Q8_3','Q8_4']].dropna(how='all', axis=1).drop_duplicates())


# take Q8_1 as trip_purp
df['trip_purp'] = df['Q8_1']
df.loc[df.trip_purp.isnull(), 'trip_purp'] = 'missing'

,Q8_1,Q8_2,Q8_3,Q8_4
0,Visit family/friends,NaN,NaN,NaN
1,Leisure/Recreation,NaN,NaN,NaN
2,Travel to/from school,NaN,NaN,NaN
3,Leisure/Recreation,Visit family/friends,NaN,NaN
4,NaN,NaN,NaN,NaN
6,Vacation,NaN,NaN,NaN
8,Other,NaN,NaN,NaN
14,Commute to/from work,NaN,NaN,NaN
23,Business travel,NaN,NaN,NaN
34,Visit family/friends,Vacation,NaN,NaN


### code home lat/lon based on zipcode <a class="anchor" id="home_lat_lon"></a>

In [12]:
# read zipcode spatial data
zip_shp = gpd.read_file(r'M:\Data\GIS layers\zip_code_sr\zip_code_sr.shp')

# get lat/lon
def getXY(pt):
    return (pt.x, pt.y)
centroidseries = zip_shp['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

zip_shp['lat'] = y
zip_shp['lon'] = x

zip_shp['postcode'] = zip_shp['postcode'].apply(lambda x: int(x))

# merge into the survey data
df = df.merge(zip_shp[['postcode', 'lat', 'lon']], left_on='Q22', right_on='postcode', how='left')

no_latlon = df.loc[df.lat.isnull() | df.lon.isnull()].shape[0]
print('{} records are missing home lat/lon, accounting for {:.1%} of all'.format(no_latlon, no_latlon/df.shape[0]))

# rename
df.rename(columns = {'lat': 'home_lat',
                     'lon': 'home_lon'}, inplace=True)

629 records are missing home lat/lon, accounting for 26.1% of all


### code board/alight station lat/lon <a class="anchor" id="station_lat_lon"></a>

In [13]:
# read x/y data
station_xy = pd.read_csv(r'M:\Data\OnBoard\Data and Reports\Capitol Corridor\OD Survey 2019\passenger_rail_stations.csv',
                         usecols = ['routename', 'station_na', 'x', 'y'])

station_xy_cc  = station_xy.loc[station_xy.routename == 'Capitol Corridor']

# rename 'Santa Clara' to 'Santa Clara Great America' and add one row for 'Santa Clara University'
station_xy_cc.loc[station_xy_cc.station_na == 'Santa Clara', 'station_na'] = 'Santa Clara Great America'

add_station = {'routename': 'Capitol Corridor', 'station_na': 'Santa Clara University', 'x': -121.9396494, 'y': 37.3517273} 
station_xy_cc = station_xy_cc.append(add_station, ignore_index = True)


# Boarding Station - for survey responses with value 21 Fairfield (Unspecified), 22 Oakland (Unspecified), 23 Santa Clara (Unspecified)
# need to re-assign to the station with more boardings within the same city

# Fairfield stations (6 and 7)
display(df.loc[(df.Q1A == 6) | (df.Q1A == 7)].Q1A.value_counts()) # 6 Suisun-fairfield has more

# Oakland stations (11 and 12)
display(df.loc[(df.Q1A == 11) | (df.Q1A == 12)].Q1A.value_counts()) # 11 Jack London Square has more

# Santa Clara stations (18 and 19)
display(df.loc[(df.Q1A == 18) | (df.Q1A == 19)].Q1A.value_counts()) # 18 Santa Clara Great America has more


# Similarly for Alighting Station

# Fairfield stations (6 and 7)
display(df.loc[(df.Q1B == 6) | (df.Q1B == 7)].Q1B.value_counts()) # 6 Suisun-fairfield has more

# Oakland stations (11 and 12)
display(df.loc[(df.Q1B == 11) | (df.Q1B == 12)].Q1B.value_counts()) # 11 Jack London Square has more

# Santa Clara stations (18 and 19)
display(df.loc[(df.Q1B == 18) | (df.Q1B == 19)].Q1B.value_counts()) # 18 Santa Clara Great America has more


# build dictionary
cc_station_dict = {'2': 'Berkeley',
                   '5': 'Emeryville',
                   '6': 'Suisun-fairfield',
                   '7': 'Fairfield/Vacaville Station',
                   '8': 'Fremont',
                   '9': 'Hayward',
                   '10': 'Martinez',
                   '11': 'Jack London Square',
                   '12': 'Oakland Coliseum',
                   '13': 'Richmond',
                   '17': 'San Jose',
                   '18': 'Santa Clara Great America',
                   '19': 'Santa Clara University',
                   '21': 'Suisun-fairfield',
                   '22': 'Jack London Square',
                   '23': 'Santa Clara Great America'}

# merge station names into the survey data
for colname in ['Q1A', 'Q1B']:
    df[colname] = df[colname].fillna(0)
    df[colname] = df[colname].apply(lambda x: str(x))
    df[colname] = df[colname].map(cc_station_dict)
    
# merge lat/lon into the survey data
df_board = df[['CCGID', 'Q1A']].merge(station_xy_cc, left_on='Q1A', right_on='station_na', how='left')
df_board = df_board[['CCGID', 'x', 'y']].rename(columns = {'x': 'survey_board_lon',
                                                        'y': 'survey_board_lat'})

df_alight = df[['CCGID', 'Q1B']].merge(station_xy_cc, left_on='Q1B', right_on='station_na', how='left')
df_alight = df_alight[['CCGID', 'x', 'y']].rename(columns = {'x': 'survey_alight_lon',
                                                          'y': 'survey_alight_lat'})

df = df.merge(df_board, on='CCGID', how='inner').merge(df_alight, on='CCGID', how='inner')

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


6    78
7    55
Name: Q1A, dtype: int64

11    268
12     81
Name: Q1A, dtype: int64

18    83
19    19
Name: Q1A, dtype: int64

6    100
7     61
Name: Q1B, dtype: int64

11    156
12     61
Name: Q1B, dtype: int64

18    128
19     35
Name: Q1B, dtype: int64

### update 'weight' <a class="anchor" id="weight"></a>

In [14]:
weight_df = pd.read_csv(r'M:\Data\OnBoard\Data and Reports\Capitol Corridor\OD Survey 2019\Weighting\Capitol_Corridor_Weights.csv')
print('read {} rows of updated weights: \n{}'.format(weight_df.shape[0], weight_df.head()))

df = df.merge(weight_df, left_on='CCGID', right_on='ID', how='left')
print('total weights: {}'.format(df.weight.sum()))
display(df.drop(columns = ['ID', 'WEIGHT'])) 

read 2406 rows of updated weights: 
   ID  weight
0   1     0.0
1   2     0.0
2   3     0.0
3   4     0.0
4   5     0.0
total weights: 5762.203065134101


,RESPNUM,CCGID,TRAIN,INTDAY,INTDATE,PERIOD,LANGUAGE,Q1A,BOARD,Q1B,...,race_dmy_mdl_estn,trip_purp,postcode,home_lat,home_lon,survey_board_lon,survey_board_lat,survey_alight_lon,survey_alight_lat,weight
0,1909,1,737,3,2019-06-22,2,2,Jack London Square,OAKLAND-JLS,San Jose,...,1,Visit family/friends,NaN,NaN,NaN,-122.271986,37.793520,-121.903040,37.329070,0.0
1,1910,2,737,3,2019-06-22,2,1,NaN,SACRAMENTO,Emeryville,...,1,Leisure/Recreation,95630.0,38.666915,-121.142106,NaN,NaN,-122.291300,37.840547,0.0
2,1911,3,737,3,2019-06-22,2,1,NaN,SACRAMENTO,Berkeley,...,1,Travel to/from school,NaN,NaN,NaN,NaN,NaN,-122.300943,37.867277,0.0
3,1912,4,737,3,2019-06-22,2,1,NaN,DAVIS,Emeryville,...,1,Leisure/Recreation,94080.0,37.655380,-122.422126,NaN,NaN,-122.291300,37.840547,0.0
4,1913,5,737,3,2019-06-22,2,1,NaN,DAVIS,Martinez,...,1,missing,94553.0,37.981739,-122.165406,NaN,NaN,-122.137097,38.019510,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,1055,2405,734,7,2019-06-29,2,1,Emeryville,EMERYVILLE,NaN,...,1,Visit family/friends,94123.0,37.800750,-122.436363,-122.291300,37.840547,NaN,NaN,0.0
2402,1057,2406,734,7,2019-06-29,2,1,Berkeley,BERKELEY,NaN,...,1,Commute to/from work,95816.0,38.575553,-121.465409,-122.300943,37.867277,NaN,NaN,0.0
2403,1060,2407,734,7,2019-06-29,2,1,Martinez,MARTINEZ,NaN,...,1,Vacation,NaN,NaN,NaN,-122.137097,38.019510,NaN,NaN,0.0
2404,1063,2408,734,7,2019-06-29,2,1,Emeryville,EMERYVILLE,NaN,...,1,Visit family/friends,95817.0,38.550905,-121.456226,-122.291300,37.840547,NaN,NaN,0.0


### impute year_born from 'age group' <a class="anchor" id="age"></a>

In [15]:
year_born_dict = {1: 2004,   # Under 18, 2019-15=2004
                  2: 1998,   # 18-24, 2019-21=1998
                  3: 1989,   # 25-34, 2019-30=1989
                  4: 1979,   # 35-44, 2019-40=1979
                  5: 1969,   # 45-54, 2019-50=1969
                  6: 1959,   # 55-64, 2019-60=1959
                  7: 1949,   # 65 and older, 2019-70=1949
                 }

df['year_born_four_digit'] = df['Q18'].map(year_born_dict)
df.year_born_four_digit.value_counts()

1989.0    520
1979.0    497
1969.0    421
1959.0    415
1949.0    209
1998.0    202
2004.0     36
Name: year_born_four_digit, dtype: int64

### export survey data <a class="anchor" id="survey_export"></a>

In [16]:
df.rename(columns = {'CCGID': 'ID'}, inplace=True)

final_fname = 'CAPCO19 Data-For MTC_NO POUND OR SINGLE QUOTE.csv'
print('export {} rows of data to {}'.format(df.shape[0], final_fname))

df.to_csv(r'M:\Data\OnBoard\Data and Reports\Capitol Corridor\OD Survey 2019\As CSV\CAPCO19 Data-For MTC_NO POUND OR SINGLE QUOTE.csv', index=False)

export 2406 rows of data to CAPCO19 Data-For MTC_NO POUND OR SINGLE QUOTE.csv


## build standard dictionary <a class="anchor" id="standard_dict"></a>

### read raw variable dictionary 'Field Guide'  <a class="anchor" id="raw_dict"></a>

In [17]:
# read raw survey dictionary
survey_dict = pd.read_excel(r'M:\Data\OnBoard\Data and Reports\Capitol Corridor\OD Survey 2019\CAPCO19 Data-For MTC.xlsx',
                            sheet_name='Field Guide')

# back fill name in 'Field' column
survey_dict.loc[(survey_dict.Field == '       ') | (survey_dict.Field == '  ') | (survey_dict.Field == '        '),
                'Field'] = np.nan
survey_dict['Field'].fillna(method='ffill', inplace=True)

# rename to correctly reflect the info
survey_dict.rename(columns={'Field': 'Survey_Variable',
                            'Question/Description': 'Survey_Response',
                            'Unnamed: 2': 'Generic_Response_old'}, inplace=True)

# only keep needed columns
var_dict = survey_dict[['Survey_Variable', 'Survey_Response', 'Generic_Response_old']].drop_duplicates()

### add rows to the dictionary for the new fields added to the survey data <a class="anchor" id="add_row"></a>

In [18]:
# add rows for 'access_mode'
access_mode_dict = var_dict.loc[(var_dict.Survey_Variable == 'Q2B_1-Q2B_6') & (var_dict.Generic_Response_old.notnull())]
access_mode_dict.loc[access_mode_dict.Survey_Variable == 'Q2B_1-Q2B_6', 'Survey_Variable'] = 'access_mode'
# display(access_mode_dict)
var_dict = var_dict.append(access_mode_dict, ignore_index=True)

# replace 'Q2A_1-Q2A_6' with new field name 'egress_mode'
var_dict.loc[(var_dict.Survey_Variable == 'Q2B_1-Q2B_6') & (var_dict.Generic_Response_old.notnull()), 'Survey_Variable'] = 'egress_mode'

# add rows for 'Q1A'
Q1A_dict = var_dict.loc[(var_dict.Survey_Variable == 'Q1B') & (var_dict.Generic_Response_old.notnull())]
Q1A_dict.loc[Q1A_dict.Survey_Variable == 'Q1B', 'Survey_Variable'] = 'Q1A'
# display(Q1A_dict)
var_dict = var_dict.append(Q1A_dict, ignore_index=True)


# add race and hispanic fields to the dictionary
race_dict = pd.DataFrame(np.array([['hispanic', 'YES', 'YES'],
                                   ['hispanic', 'NO', 'NO'],
                                   ['race_dmy_ind', 1, 1],
                                   ['race_dmy_hwi', 1, 1],
                                   ['race_dmy_blk', 1, 1],
                                   ['race_dmy_wht', 1, 1],
                                   ['race_dmy_asn', 1, 1],
                                   ['race_dmy_mdl_estn', 1, 1],
                                   ['race_dmy_ind', 0, 0],
                                   ['race_dmy_hwi', 0, 0],
                                   ['race_dmy_blk', 0, 0],
                                   ['race_dmy_wht', 0, 0],
                                   ['race_dmy_asn', 0, 0],
                                   ['race_dmy_mdl_estn', 0, 0]]),
                         columns=['Survey_Variable', 'Survey_Response', 'Generic_Response_old'])
# display(race_dict)
var_dict = var_dict.append(race_dict, ignore_index=True)


# add trip_purp rows
trip_purpose_dict = df[['trip_purp']].drop_duplicates()
trip_purpose_dict.columns = ['Survey_Response']
trip_purpose_dict['Survey_Variable'] = 'trip_purp'
trip_purpose_dict['Generic_Response_old'] = trip_purpose_dict['Survey_Response']
# trip_purpose_dict[['Survey_Variable', 'Survey_Response', 'Generic_Response_old']]
var_dict = var_dict.append(trip_purpose_dict, ignore_index=True)


# add home lat/lon rows
home_dict = pd.DataFrame(np.array([['home_lat', 'NONCATEGORICAL', np.nan],
                                   ['home_lon', 'NONCATEGORICAL', np.nan]]),
                         columns=['Survey_Variable', 'Survey_Response', 'Generic_Response_old'])
var_dict = var_dict.append(home_dict, ignore_index=True)


# add survey_board/alight lat/lon rows
board_alight_dict = pd.DataFrame(np.array([['survey_board_lon', 'NONCATEGORICAL', np.nan],
                                           ['survey_board_lat', 'NONCATEGORICAL', np.nan],
                                           ['survey_alight_lon', 'NONCATEGORICAL', np.nan],
                                           ['survey_alight_lat', 'NONCATEGORICAL', np.nan]]),
                                 columns=['Survey_Variable', 'Survey_Response', 'Generic_Response_old'])
var_dict = var_dict.append(board_alight_dict, ignore_index=True)


# add transfer routes rows
trans_routes_dict = pd.DataFrame(np.array([['first_route_before_survey_board', 'NONCATEGORICAL', np.nan],
                                           ['second_route_before_survey_board', 'NONCATEGORICAL', np.nan],
                                           ['third_route_before_survey_board', 'NONCATEGORICAL', np.nan],
                                           ['first_route_after_survey_alight', 'NONCATEGORICAL', np.nan],
                                           ['second_route_after_survey_alight', 'NONCATEGORICAL', np.nan],
                                           ['third_route_after_survey_alight', 'NONCATEGORICAL', np.nan]]),
                                 columns=['Survey_Variable', 'Survey_Response', 'Generic_Response_old'])
var_dict = var_dict.append(trans_routes_dict, ignore_index=True)

# add new weight row
var_dict.loc[len(var_dict.index)] = ['weight', 'NONCATEGORICAL', np.nan]

# add year_born row
var_dict.loc[len(var_dict.index)] = ['year_born_four_digit', 'NONCATEGORICAL', np.nan]

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


### add default fields in the standard dictionary <a class="anchor" id="add_fields"></a>

In [19]:
# add columns 'Generic_variable'
var_dict['Generic_Variable'] = ''
var_dict.loc[var_dict.Survey_Variable == 'CCGID', 'Generic_Variable'] = 'ID'
var_dict.loc[var_dict.Survey_Variable == 'CCGID', 'Survey_Variable'] = 'ID'

var_dict.loc[var_dict.Survey_Variable == 'TRAIN', 'Generic_Variable'] = 'route'
var_dict.loc[var_dict.Survey_Variable == 'INTDATE', 'Generic_Variable'] = 'date_string'
var_dict.loc[var_dict.Survey_Variable == 'PERIOD', 'Generic_Variable'] = 'weekpart'
var_dict.loc[var_dict.Survey_Variable == 'Language', 'Generic_Variable'] = 'interview_language'
var_dict.loc[var_dict.Survey_Variable == 'Language', 'Survey_Variable'] = 'LANGUAGE'

var_dict.loc[var_dict.Survey_Variable == 'Q9', 'Generic_Variable'] = 'fare_category'
var_dict.loc[var_dict.Survey_Variable == 'Q10', 'Generic_Variable'] = 'fare_medium'

var_dict.loc[var_dict.Survey_Variable == 'Q17', 'Generic_Variable'] = 'gender'
var_dict.loc[var_dict.Survey_Variable == 'Q19', 'Generic_Variable'] = 'household_income'

var_dict.loc[var_dict.Survey_Variable == 'Q21', 'Generic_Variable'] = 'persons'


# newly created fields:
var_dict.loc[var_dict.Survey_Variable.str.contains('race_dmy_',na=False), 'Generic_Variable'] = var_dict['Survey_Variable']

for varname in ['access_mode', 'egress_mode', 'trip_purp', 'home_lat', 'home_lon', 'weight', 'hispanic',
                'survey_board_lon', 'survey_board_lat', 'survey_alight_lon', 'survey_alight_lat',
                'first_route_before_survey_board', 'second_route_before_survey_board',
                'third_route_before_survey_board', 'first_route_after_survey_alight',
                'second_route_after_survey_alight', 'third_route_after_survey_alight']:
    var_dict.loc[var_dict.Survey_Variable == varname,  'Generic_Variable'] = varname

In [20]:
# add values for 'Survey_Response' and 'Generic_Response' for noncanonical variables
var_dict['Generic_Response'] = ''

for varname in ['ID', 'TRAIN', 'INTDATE', 'Q22', 'weight']:
    var_dict.loc[var_dict.Survey_Variable == varname, 'Survey_Response'] = 'NONCATEGORICAL'
    
for varname in ['ID', 'TRAIN', 'INTDATE', 'Q22', 'weight', 'year_born_four_digit',
                'home_lat', 'home_lon',
                'survey_board_lon', 'survey_board_lat', 'survey_alight_lon', 'survey_alight_lat',
                'first_route_before_survey_board', 'second_route_before_survey_board',
                'third_route_before_survey_board', 'first_route_after_survey_alight',
                'second_route_after_survey_alight', 'third_route_after_survey_alight']:
    var_dict.loc[var_dict.Survey_Variable == varname, 'Generic_Response'] = 'NONCATEGORICAL'


In [21]:
var_dict['operator'] = 'Capitol Corridor'
var_dict['Survey_year'] = 2019

### check consistency between values in survey data and in the dictionary <a class="anchor" id="check"></a>

In [22]:
# check if all values in the survey data are represented in the dictionary

var_dict_cat = var_dict.loc[(var_dict.Survey_Response != 'NONCATEGORICAL') & (
                             var_dict.Generic_Response_old.notnull()) & (
                             var_dict.Generic_Variable != '')]

for fieldname in list(var_dict_cat.Survey_Variable.unique()):
#     fieldname = fieldname.upper()
    print(fieldname)
    values = var_dict.loc[var_dict.Survey_Variable == fieldname]
    
    if fieldname in ['race_dmy_ind', 'race_dmy_hwi', 'race_dmy_blk', 'race_dmy_wht', 'race_dmy_asn', 'race_dmy_mdl_estn']:
        values.Survey_Response = values.Survey_Response.apply(lambda x: int(x))
        
    comp = df.merge(values, left_on = fieldname, right_on = 'Survey_Response', how='left')
    comp_diff = comp.loc[comp[fieldname].isnull()]
    if comp_diff.shape[0] > 0:
        print(comp_diff[fieldname].unique())

PERIOD
LANGUAGE
egress_mode
Q9
Q10
Q17
Q19
Q21
access_mode
hispanic
race_dmy_ind
race_dmy_hwi
race_dmy_blk
race_dmy_wht
race_dmy_asn
race_dmy_mdl_estn
trip_purp


C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


### export raw standard dictionary <a class="anchor" id="export_dict"></a>

In [23]:
# will manually add 'Generic_Response' for canonical variables in the exported file

# only keep rows for needed variables
var_dict = var_dict.loc[(var_dict.Generic_Variable != '') & (
    (var_dict.Generic_Response == 'NONCATEGORICAL') | var_dict.Generic_Response_old.notnull())]

print('export raw dictionary with {} variables:\n{}'.format(len(var_dict.Survey_Variable.unique()),
                                                          var_dict.Survey_Variable.unique()))

var_dict.to_csv(r'M:\Data\OnBoard\Data and Reports\Capitol Corridor\OD Survey 2019\var_dict_raw.csv', index=False)

export raw dictionary with 33 variables:
['ID' 'TRAIN' 'INTDATE' 'PERIOD' 'LANGUAGE' 'egress_mode' 'Q9' 'Q10' 'Q17'
 'Q19' 'Q21' 'access_mode' 'hispanic' 'race_dmy_ind' 'race_dmy_hwi'
 'race_dmy_blk' 'race_dmy_wht' 'race_dmy_asn' 'race_dmy_mdl_estn'
 'trip_purp' 'home_lat' 'home_lon' 'survey_board_lon' 'survey_board_lat'
 'survey_alight_lon' 'survey_alight_lat' 'first_route_before_survey_board'
 'second_route_before_survey_board' 'third_route_before_survey_board'
 'first_route_after_survey_alight' 'second_route_after_survey_alight'
 'third_route_after_survey_alight' 'weight']


## build canonical route crosswalk <a class="anchor" id="canonical_route"></a>

In [24]:
canonical_routes = pd.DataFrame(np.array([[5, 'BART___BART',             'BART',     'heavy rail'],
                                          [6, 'CALTRAIN___CALTRAIN',     'Caltrain', 'commuter rail'],
                                          [7, 'Missing___missing',       'Missing',  'light rail'],
                                          [8, 'AMTRAK___Amtrak Shuttle', 'AMTRAK',   'local bus'],
                                          [9, 'AMTRAK___AMTRAK',         'AMTRAK',   'commuter rail'],
                                          [10,'Missing___missing',       'Missing',  'local_bus']]),
                                columns=['survey_name','canonical_name','canonical_operator','technology'])

canonical_routes['survey'] = 'Capitol Corridor'
canonical_routes['survey_year'] = 2019
canonical_routes.to_csv(r'M:\Data\OnBoard\Data and Reports\Capitol Corridor\OD Survey 2019\routes_canonical.csv', index=False)